# Tutorial 07: Open shells

In this tutorial you will learn:
- Examing the difference between restricted and unrestricted computations
- Learn how  psi4 handles open-shell species

In [ ]:
import psi4

When doing Hartree–Fock or DFT computations on closed-shell species (even number of electrons, all paired) **psi4 automatically selectes a restricted formalism**. This means that the spatial part of alpha and beta orbitals is the same
$$
\phi_{i\alpha}(\mathbf{r}) = \phi_{i\beta}(\mathbf{r})
$$
and the orbital energies are the same
$$
\epsilon_{i\alpha} = \epsilon_{i\beta}
$$

Let's look how this restriction is reflected in a simple example:

In [ ]:
psi4.geometry(f"""
H
F 1 1.0
symmetry c1
""")

e, wfn = psi4.energy('SCF/def2-SVP',return_wfn=True)

For closed-shell specied, a restricted formalism give the identical alpha and beta orbital energies

In [ ]:
print('epsilon_a = ',wfn.epsilon_a().to_array()) # alpha orbital energies
print('epsilon_a = ',wfn.epsilon_b().to_array()) # beta orbital energies
print('epsilon_a - epsilon_b = ',wfn.epsilon_a().to_array() - wfn.epsilon_b().to_array()) # alpha-beta orbital energy difference

Recall that the molecular orbitals are a linear combination of the atomic orbital basis ($\chi_\mu$) times a coefficient matrix ($C_{\mu i}$).
Here we write both the alpha and beta components explicitly
$$
\phi_{i\alpha}(\mathbf{r}) = \sum_\mu^{\mathrm{basis}} \chi_\mu(\mathbf{r}) C_{\mu i}^{\alpha}
$$
$$
\phi_{i\beta}(\mathbf{r}) = \sum_\mu^{\mathrm{basis}} \chi_\mu(\mathbf{r}) C_{\mu i}^{\beta}
$$

For a closed-shell molecule, we can easily verify that $C_{\mu i}^{\alpha}$ and $C_{\mu i}^{\beta}$ are identical since their difference is a matrix of zeros:

In [ ]:
print(wfn.Ca().to_array() - wfn.Cb().to_array()) # alpha orbital energies

## Simple open shells

Suppose that we are now interested in studying the H$_2^+$ molecule. This system has one unpaired electron, which implies that the spin of the H$_2^+$ molecule is nonzero.
A single electron is characterized by two quantum numbers:
- The total spin quantum number $s = 1/2$ (corresponding to the operator $\hat{S}^2$), which tells us that the average total spin squared is 
$$
S^2 = s(s+1) = \frac{3}{4}
$$
- The z component of spin quantum number $m_s =$ for which we have two possible choices
$$
m_s = +\frac{1}{2} \equiv \alpha \text{   or   } -\frac{1}{2}  \equiv \beta.
$$
The average value of the spin projected on the z axis is
$$
S_z = m_s
$$

Since there are two possible states for the spin of the electron, we say that this system has **multiplicity equal to 2**. Another way to compute the multiplicity is via the equation
$$
\text{multiplicity } = 2s + 1
$$
Let's now try to compute the Hartree–Fock energy of H$_2^+$ with psi4

## Restricted vs. unrestricted methods

There are two ways to perform computations on open shell systems like H$_2^+$. In every computation we need to specify one.

1. using a restricted formalism that enforces the condition
$$
\phi_{i\alpha}(\mathbf{r}) = \phi_{i\beta}(\mathbf{r})
$$
for all doubly occupied orbitals.
1. using an unrestricted formalism in which we let alpha and beta orbitals have different spatial part
$$
\phi_{i\alpha}(\mathbf{r}) \neq \phi_{i\beta}(\mathbf{r})
$$
for both doubly and singly occupied orbitals.


Note that **for a single electron, these two approaches give the same energy**.

To perform a restricted Hartree–Fock computation you neet to set the option `REFERENCE` to `ROHF`.

In [ ]:
psi4.geometry("""
1 2
H
H 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'ROHF'})


e, wfn = psi4.energy('HF/def2-SVP',return_wfn=True)

Now let us check the orbital energies. In the case of ROHF these might be defined differently by different programs because there is not a unique way to define them. Psi4 yields equal energies for the alpha and beta orbitals

In [ ]:
print(wfn.epsilon_a().to_array()) # alpha orbital energies
print(wfn.epsilon_b().to_array()) # beta orbital energies
print(wfn.Ca().to_array() - wfn.Cb().to_array()) # alpha orbital energies

An unrestricted HF computation can be done by setting the option `REFERENCE` to `UHF`.

In [ ]:
psi4.geometry("""
1 2
H
H 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'UHF'})

e, wfn = psi4.energy('HF/def2-SVP',return_wfn=True)
print(wfn.epsilon_a().to_array()) # alpha orbital energies
print(wfn.epsilon_b().to_array()) # beta orbital energies
print(wfn.Ca().to_array() - wfn.Cb().to_array()) # alpha orbital energies

Note that the energies of the alpha and beta electrons are different. In the unrestricted case the alpha and beta orbitals are different and we can see this if we plot them.

The following example shows the difference in the UHF orbitals of LiH+

In [ ]:
psi4.geometry("""
1 2
Li
H 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'UHF'})
e, wfn = psi4.energy('HF/def2-SVP',return_wfn=True)

psi4.cubeprop(wfn)

import fortecubeview
fortecubeview.plot()

Note that **in UHF the orbitals may be different but this is not always the case**. For example, if you apply UHF to a closed shell system then often you recover the RHF solution. For example

In [ ]:
psi4.geometry("""
0 1
H
F 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'UHF'})
e_uhf = psi4.energy('HF/def2-SVP')

psi4.set_options({'REFERENCE' : 'RHF'})
e_rhf = psi4.energy('HF/def2-SVP')

print(f'The difference between the RHF and UHF energy is {e_rhf - e_uhf}')

## General case

When we have more than one electron, we use capital letters to refer to the sum of the angular momentum of all electrons. The quantum number $M_S$ is the sum of the z component of spin for all the electrons, which is given by the difference between the number of alpha and beta electrons divided by 2
$$
M_S = \frac{1}{2} (N_\alpha - N_\beta)
$$

An important result in the theory of angular momentum is that a system with total spin equal to $S$ has multiplicity = $2S + 1$, which means that there are $2S + 1$ states with the same total spin. These states have z-projection of spin that ranges from $-S$ to $S$ in increments of 1
$$
-S, -S + 1, -S + 2, \ldots, S - 2, S-1, S
$$

Now consider the case of HF$^+$. This molecule has 5 alpha and 4 beta electrons (or the other way around).
For HF$^+$ has $M_S = 1/2$ which comes from the 5 alpha and 4 beta electrons ((5 - 4)/2).

For HF$^+$ this means that the state with one unpaired alpha electron corresponds to $S = 1/2$. If it was otherwise things would not work out. For example, if $S = 1$ then we should find state with 
$$
M_S = -1, 0, +1
$$
but we know that with one unpaired electron we need to have a state with $M_S = 1/2$.
If we postulated that $S = 3/2$ then we should be able to find states with
$$
M_S = -3/2, -1/2, 1/2 , 3/2
$$
However, with only one unpaired electron we cannot have a state with $M_S = 3/2$.

To summarize, the HF$^+$ molecule has multiplicity equal to $2S + 1 = 2 1/2 + 1 = 2$.

## Spin contamination of unrestricted wave functions

ROHF and UHF generally yield different energies

In [ ]:
psi4.geometry("""
1 2
H
F 1 1.0
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'ROHF'})
e_rohf = psi4.energy('HF/def2-SVP')

psi4.set_output_file('uhf.txt',False)

psi4.set_options({'REFERENCE' : 'UHF'})
e_uhf = psi4.energy('HF/def2-SVP')

print(f'E(ROHF) = {e_rohf} Eh')
print(f'E(UHF)  = {e_uhf}  Eh')
print(f'The difference between the ROHF and UHF energy is {e_rohf - e_uhf}')

Why is there an energy difference between the ROHF and UHF? And why is UHF the lower solution? Let's check the output

In [ ]:
with open('uhf.txt') as f:
    print(''.join(f.readlines()[130:]))

## Spin density of methylene computed with DFT

In [ ]:
import psi4 
psi4.geometry(f"""
0 3
C
H 1 1.0
H 1 1.0 2 104.5
symmetry c1
""")

psi4.set_options({'REFERENCE' : 'UKS'})
e, wfn = psi4.optimize('B3LYP/def2-SVP',return_wfn=True)

In [ ]:
!mkdir ch2
psi4.set_options({'CUBEPROP_FILEPATH' : './ch2','CUBEPROP_TASKS' : ['DENSITY']})
psi4.cubeprop(wfn)

In [ ]:
!ls ch2
# !rm ch2/Ps*

import fortecubeview
fortecubeview.plot(path='./ch2',sumlevel=0.9)